# Preparation

## Load bibs

In [ ]:
# Standard libraries
#import os
import pickle
from collections import Counter

# Third-party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib import gridspec

# Sklearn modules
from sklearn.cluster import DBSCAN
from sklearn.ensemble import RandomForestRegressor
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.model_selection import ParameterGrid


In [ ]:
from my_functions.functions_tsne_analysis import DataFrameProcessor   # Der Name der Python-Datei ohne die Endung .py

## Load data

### Shape files

In [ ]:
state_shape = pd.read_pickle('data/original_data/pkl/state.pickle')
county_shape = pd.read_pickle('data/original_data/pkl/county.pickle')
county_shape = county_shape.rename(columns={'GEOID': 'FIPS'})

filtered_statefp_north_america = state_shape.loc[~state_shape['NAME'].isin(['Alaska', 'Hawaii', 'Puerto Rico', 'Commonwealth of the Northern Mariana Islands', 'American Samoa', 'United States Virgin Islands', 'Guam']), 'STATEFP']
filtered_statefp_alaska = state_shape.loc[state_shape['NAME'].isin(['Alaska']), 'STATEFP']


filtered_state_shape_north_america = state_shape[state_shape['STATEFP'].isin(filtered_statefp_north_america)]
filtered_state_shape_alaska = state_shape[state_shape['STATEFP'].isin(filtered_statefp_alaska)]


filtered_county_shape_north_america = county_shape[county_shape['STATEFP'].isin(filtered_statefp_north_america)]
filtered_county_shape_alaska = county_shape[county_shape['STATEFP'].isin(filtered_statefp_alaska)]

### Feature dataframe

In [ ]:
df_original = pd.read_pickle('data/processed_data/pkl/final_df.pickle')
print(df_original['FIPS'].nunique())
df_original

In [ ]:
# Select and copy the columns to drop, preserving them for future use or reference.
# Drop the selected columns from the original DataFrame and preprocess the remaining features.
columns_to_drop = ['FIPS']
df_ids = df_original[columns_to_drop].copy()

# Entfernen der Spalten aus dem ursprünglichen DataFrame
df_features = df_original.drop(columns=columns_to_drop)
df_features, replacement_values, min_values = DataFrameProcessor.replace_nulls_with_small_values(df_features)

df_scaled_features = np.log10(df_features)
df_original_features = df_original.iloc[:, 1:]

df_scaled = pd.concat([df_ids, df_scaled_features], axis=1)

In [ ]:
# Extract column names from both original and scaled DataFrames for comparison and visualization.
# Create subplots to compare histograms of original and scaled data for each column.

columns_original = df_original.columns[1:]
columns_scaled = df_scaled.columns[1:]
n_columns = len(columns_original)  # Number of columns (assuming both DataFrames have the same number of columns)

# Create subplots: 2 columns (one for df_original and one for df_scaled_features)
fig, axs = plt.subplots(n_columns, 2, figsize=(12, 4*n_columns))

# Plot a histogram for each column in df_original and df_scaled_features
for i, col in enumerate(columns_original):
    
    # Plot histogram for df_original
    ax_features = axs[i, 0]
    min_val_features = df_original[col].min()
    max_val_features = df_original[col].max()
    bins_features = np.linspace(min_val_features, max_val_features, 51)
    ax_features.hist(df_original[col], bins=bins_features, alpha=0.5, label=col, density=True)
    ax_features.set_title(f'Histogram of {col} (df_original)')
    ax_features.set_xlabel(col)
    ax_features.set_ylabel('Density')
    ax_features.legend()

    # Plot histogram for df_scaled_features if the column exists
    ax_scaled = axs[i, 1]
    if col in columns_scaled:
        min_val_scaled = df_scaled_features[col].min()
        max_val_scaled = df_scaled_features[col].max()
        bins_scaled = np.linspace(min_val_scaled, max_val_scaled, 51)
        ax_scaled.hist(df_scaled_features[col], bins=bins_scaled, alpha=0.5, label=col, density=True)
        ax_scaled.set_title(f'Histogram of {col} (df_scaled)')
        ax_scaled.set_xlabel(col)
        ax_scaled.set_ylabel('Density')
        ax_scaled.legend()

# Adjust layout for better visualization
plt.tight_layout()
plt.show()

In [ ]:
# Create a dictionary mapping each feature to a unique identifier in the format "F1", "F2", etc.
# This helps simplify feature naming and improves readability in further processing or analysis.
feature_id_dict = {feature: "F" + str(i+1) for i, feature in enumerate(df_original_features.columns)}
feature_id_dict =  0

# Analysis

In [ ]:
# Initialize a t-SNE model with specific parameters for dimensionality reduction.
# This will be applied to the unfiltered dataset to project high-dimensional features into a 2D space for visualization.
tsne_3 = TSNE(n_components=2, perplexity=40, learning_rate=200, n_iter=500, random_state=42)
X_tsne_3 = tsne_3.fit_transform(df_scaled_features)

In [ ]:
# Create a scatter plot to visualize the t-SNE results for the third dataset.
# The plot displays the two t-SNE components on the x- and y-axes, providing insight into the structure of the data.
plt.figure(figsize=(8, 6))
scatter_3 = plt.scatter(X_tsne_3[:, 0], X_tsne_3[:, 1], s=5)
plt.title('t-SNE Visualization')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')

# Adjust layout for better spacing and display the plot.
plt.tight_layout()
plt.show()

## Cluster identification

This process involves reducing the dimensionality of a high-dimensional dataset using t-SNE and then clustering the data with DBSCAN. 
The t-SNE step transforms complex data into two dimensions, capturing key structure and relationships in a way that’s visually interpretable. 
**Afterward, DBSCAN identifies clusters based on density, distinguishing between core clusters and noise points.** 
The resulting clusters are then visualized in a scatter plot, with color-coded points representing different clusters and noise, allowing for easy interpretation of the data's underlying patterns and structure.

In [ ]:
# Define parameters for the grid search
param_grid = {
    'eps': np.arange(2.0, 4.1, 0.5),      # Example range for eps values
    'min_samples': np.arange(20, 36, 5), # Example range for min_samples values
}

# List to store the results of the grid search
grid_results = []

# Iterate through all parameter combinations in the grid search
for params in ParameterGrid(param_grid):
    # Initialize DBSCAN with the current parameters
    dbscan = DBSCAN(eps=params['eps'], min_samples=params['min_samples'])
    cluster_labels = dbscan.fit_predict(X_tsne_3)

    # Count clusters and noise points
    cluster_counts = Counter(cluster_labels)
    total_points = len(cluster_labels)
    noise_count = cluster_counts[-1] if -1 in cluster_counts else 0
    noise_ratio = noise_count / total_points

    # Check if more than one cluster was found and noise is below 10%
    if len(set(cluster_labels)) > 1 and noise_ratio <= 0.1:
        # Compute the silhouette score
        silhouette_avg = silhouette_score(X_tsne_3, cluster_labels)
        
        # Compute silhouette coefficients for each point
        silhouette_vals = silhouette_samples(X_tsne_3, cluster_labels)
        
        # Count points with negative silhouette coefficients
        negative_silhouette_count = np.sum(silhouette_vals < 0)

        # Save the result
        grid_results.append({
            'eps': params['eps'],
            'min_samples': params['min_samples'],
            'silhouette_avg': silhouette_avg,
            'negative_silhouette_count': negative_silhouette_count,
            'noise_ratio': noise_ratio
        })

        # Silhouette coefficients for each cluster
        unique_labels = set(cluster_labels)
        cluster_silhouette = {label: np.mean(silhouette_vals[cluster_labels == label]) 
                              for label in unique_labels if label != -1}

        # Define colors for clusters
        colors = plt.cm.Paired(np.linspace(0, 1, len(unique_labels) + 1))  # +1 for noise
        color_dict = {label: "#{:02x}{:02x}{:02x}".format(int(c[0]*255), int(c[1]*255), int(c[2]*255)) 
                      for label, c in zip(unique_labels, colors)}
        color_dict[-1] = "#000000"  # Black for noise

        # Create figure with two side-by-side plots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

        # Create t-SNE scatter plot with cluster coloring
        for label in unique_labels:
            if label == -1:
                ax1.scatter(X_tsne_3[cluster_labels == label, 0], X_tsne_3[cluster_labels == label, 1],
                            color=color_dict[label], label=f'Noise ({cluster_counts[label]})', alpha=0.6, s=4)
            else:
                ax1.scatter(X_tsne_3[cluster_labels == label, 0], X_tsne_3[cluster_labels == label, 1],
                            color=color_dict[label], label=f'Cluster {label} ({cluster_counts[label]})', alpha=0.6, s=4)

        ax1.set_title(f't-SNE with DBSCAN Clustering\n(eps={params["eps"]}, min_samples={params["min_samples"]})')
        ax1.set_xlabel('t-SNE 1')
        ax1.set_ylabel('t-SNE 2')
        ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3)

        # Silhouette visualization for clusters
        y_lower = 10
        for cluster in unique_labels:
            if cluster != -1:
                cluster_vals = silhouette_vals[cluster_labels == cluster]
                cluster_vals.sort()
                size_cluster = cluster_vals.shape[0]
                ax2.fill_betweenx(np.arange(y_lower, y_lower + size_cluster),
                                  0, cluster_vals, color=color_dict[cluster])
                ax2.text(-0.05, y_lower + size_cluster / 2, str(cluster))
                y_lower += size_cluster

        ax2.set_title("Silhouette Plot for All Clusters")
        ax2.set_xlabel("Silhouette Coefficient")
        ax2.set_ylabel("Cluster")
        ax2.axvline(x=silhouette_avg, color="red", linestyle="--")

        plt.tight_layout()
        plt.show()

        # Print silhouette results
        print("-" * 100)
        print(f"Parameters: eps={params['eps']}, min_samples={params['min_samples']}")
        print(f"Average Silhouette Coefficient: {silhouette_avg}")
        print(f"Number of Points with Negative Silhouette Coefficients: {negative_silhouette_count}")
        print(f"Number of Noise Points: {noise_count}")
        print(f"Noise Ratio: {noise_ratio:.2%}")
        for cluster, score in cluster_silhouette.items():
            print(f"Average Silhouette Coefficient for Cluster {cluster}: {score:.3f}")
        print("-" * 100)

# Total number of points and noise points
total_points = len(grid_results)  # Total number of points
noise_threshold = 0.05  # 5% threshold for noise

# Find the best result
best_result = max(
    grid_results,
    key=lambda x: (
        x['silhouette_avg'],
        -x['negative_silhouette_count'] if (x['negative_silhouette_count'] / total_points) < noise_threshold else float('-inf')
    )
)

# Print the best result
print("Best Parameter Combination:")
print(f"eps: {best_result['eps']}, min_samples: {best_result['min_samples']}")
print(f"Average Silhouette Coefficient: {best_result['silhouette_avg']}")
print(f"Number of Points with Negative Silhouette Coefficients: {best_result['negative_silhouette_count']}")
print(f"Noise Ratio: {best_result['noise_ratio']:.2%}")


In [ ]:
# Create and fit DBSCAN clusterer
dbscan = DBSCAN(eps=3, min_samples=25)
cluster_labels = dbscan.fit_predict(X_tsne_3)
cluster_counts = Counter(cluster_labels)


# Create color assignments
unique_labels = np.unique(cluster_labels)
colors = plt.cm.Dark2(np.linspace(0, 1, len(unique_labels) + 1))  # +1 for noise

# Dictionary to map colors to cluster IDs using color strings
color_dict = {label: "#{:02x}{:02x}{:02x}".format(int(c[0]*255), int(c[1]*255), int(c[2]*255)) for i, (label, c) in enumerate(zip(unique_labels, colors))}
color_dict[-1] = "#000000"  # Black for "Noise"

# Add a new column for colors to the DataFrame
df_original['cluster_id'] = cluster_labels
df_scaled['cluster_id'] = cluster_labels

df_original['color'] = df_original['cluster_id'].map(color_dict)
df_scaled['color'] = df_scaled['cluster_id'].map(color_dict)


# Count the number of "Noise" points
noise_count = cluster_counts[-1]
print(f'Number of "Noise" points: {noise_count}')

# Plot for the third dataset with clusters
plt.figure(figsize=(8, 6))

# Scatter plot with cluster colors
for label in unique_labels:
    if label == -1:
        # -1 represents noise, so plot in black
        plt.scatter(X_tsne_3[cluster_labels == label, 0], X_tsne_3[cluster_labels == label, 1],
                    color=color_dict[label], label=f'Noise ({noise_count})', alpha=0.6, s=5)
    else:
        plt.scatter(X_tsne_3[cluster_labels == label, 0], X_tsne_3[cluster_labels == label, 1],
                    color=color_dict[label], label=f'Cluster {label} ({cluster_counts[label]})', alpha=0.6, s=5)

plt.title('t-SNE Visualization with DBSCAN Clustering')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')

# Add legend and position it below the plot
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()


## t-SNE with heatmap for every single feature

In [ ]:
# Beispiel DataFrame (X_unfiltered_copy) erstellen
# X_unfiltered_copy = pd.DataFrame(...) # Dein DataFrame hier


# Iteriere über die Spalten, um Heatmaps zu erstellen
for col in df_scaled_features:
    plt.figure(figsize=(8, 5))
    scatter = plt.scatter(X_tsne_3[:, 0], X_tsne_3[:, 1], c=df_scaled_features[col], cmap='viridis', alpha=0.7, s =5)
    
    # Farbskala hinzufügen
    plt.colorbar(scatter, label=col)
    plt.title(f't-SNE Visualisierung für {col}')
    plt.xlabel('t-SNE 1')
    plt.ylabel('t-SNE 2')

    # Layout anpassen und anzeigen
    plt.tight_layout()
    plt.show()


## Cluster map visualization

In [ ]:
filtered_county_shape_north_america_merged = filtered_county_shape_north_america.merge(df_scaled[['FIPS', 'cluster_id','color']], on='FIPS', how='left')
filtered_county_shape_north_america_merged.head(3)

In [ ]:
filtered_county_shape_alaska_merged = filtered_county_shape_alaska.merge(df_scaled[['FIPS', 'cluster_id','color']], on='FIPS', how='left')
filtered_county_shape_alaska_merged.head(3)

In [ ]:
# Create the plot.
fig, ax = plt.subplots(figsize=(15, 7))  # Initialize a figure and axes for the plot.
filtered_county_shape_north_america_merged.plot(facecolor=filtered_county_shape_north_america_merged['color'], linewidth=0.8, ax=ax, edgecolor='0.8', legend=False)  # Plot counties with assigned colors.

# Overlay county and state outlines
filtered_county_shape_north_america.plot(ax=ax, color='none', edgecolor='gray', linewidth=0.4, alpha=0.5)  # Overlay county outlines.
filtered_state_shape_north_america.plot(ax=ax, color='none', edgecolor='black', linewidth=0.5)  # Overlay state outlines.

# Create a custom legend for the clusters.
unique_clusters = sorted(filtered_county_shape_north_america_merged['cluster_id'].unique())  # Get unique cluster IDs
cluster_counts = filtered_county_shape_north_america_merged['cluster_id'].value_counts()  # Count the number of entries per cluster
legend_elements = []

# Loop through each cluster and create legend elements
for cluster in unique_clusters:
    if cluster == -1:
        # For cluster -1, use gray color and label as 'noise'
        count = cluster_counts.get(-1, 0)
        legend_elements.append(Patch(facecolor='black', label=f'noise ({count})'))
    else:
        color = filtered_county_shape_north_america_merged.loc[filtered_county_shape_north_america_merged['cluster_id'] == cluster, 'color'].values[0]
        count = cluster_counts.get(cluster, 0)
        legend_elements.append(Patch(facecolor=color, label=f'{cluster} ({count})'))  # Add count to the legend label

# Add the legend to the plot.
ax.legend(handles=legend_elements, title='Cluster ID', loc='lower right')  # Position the legend in the lower right corner.

plt.title('Cluster Assignment to Counties')  # Set the plot title.
plt.axis('off')  # Turn off the axes for a cleaner look.
plt.show()  # Display the plot.


In [ ]:
# Create a figure with a specific grid layout
fig = plt.figure(figsize=(20, 10))
gs = gridspec.GridSpec(1, 2, width_ratios=[65, 35])  # Set width ratios for subplots

# Create subplots using the grid spec
ax1 = fig.add_subplot(gs[0])  # North America subplot
ax2 = fig.add_subplot(gs[1])  # Alaska subplot

# Plot for North America
filtered_county_shape_north_america_merged.plot(
    facecolor=filtered_county_shape_north_america_merged['color'], 
    linewidth=0.8, 
    ax=ax1, 
    edgecolor='0.8', 
    legend=False
)  # Plot counties with assigned colors.

# Overlay county and state outlines for North America
filtered_county_shape_north_america.plot(
    ax=ax1, 
    color='none', 
    edgecolor='gray', 
    linewidth=0.4, 
    alpha=0.5
)  # Overlay county outlines.
filtered_state_shape_north_america.plot(
    ax=ax1, 
    color='none', 
    edgecolor='black', 
    linewidth=0.5
)  # Overlay state outlines.

# Create a custom legend for the clusters
unique_clusters = sorted(filtered_county_shape_north_america_merged['cluster_id'].unique())  # Get unique cluster IDs
cluster_counts = filtered_county_shape_north_america_merged['cluster_id'].value_counts()  # Count the number of entries per cluster
legend_elements = []

# Loop through each cluster and create legend elements
for cluster in unique_clusters:
    if cluster == -1:
        # For cluster -1, use gray color and label as 'noise'
        count = cluster_counts.get(-1, 0)
        legend_elements.append(Patch(facecolor='black', label=f'noise ({count})'))
    else:
        color = filtered_county_shape_north_america_merged.loc[filtered_county_shape_north_america_merged['cluster_id'] == cluster, 'color'].values[0]
        count = cluster_counts.get(cluster, 0)
        legend_elements.append(Patch(facecolor=color, label=f'{cluster} ({count})'))  # Add count to the legend label

# Add the legend to the plot
ax1.legend(handles=legend_elements, title='Cluster ID', loc='lower right')  # Position the legend in the lower right corner.
ax1.set_title('North America County Clusters')
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')

# Plot for Alaska
# Assuming filtered_county_shape_alaska has a 'color' column based on clusters
filtered_county_shape_alaska_merged.plot(
    facecolor=filtered_county_shape_alaska_merged['color'], 
    linewidth=0.5, 
    ax=ax2, 
    edgecolor='black'
)  # Plot counties in Alaska with assigned colors.

# Overlay state outlines for Alaska
filtered_county_shape_alaska_merged.plot(
    ax=ax2, 
    color='none', 
    edgecolor='black', 
    linewidth=0.5
)  # Overlay state outlines for Alaska.

# Set limits and labels for Alaska
ax2.set_xlim(-180, -130)
ax2.set_ylim(50, 72)
ax2.set_title('Alaska County Clusters')
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')

# Adjust layout
plt.tight_layout()

# Display the combined plot
plt.show()


## Feature characteristics for each cluster

In [ ]:
# Select only the numeric columns that are not 'cluster_id', 'color', 'FIPS'
numeric_columns = [col for col in df_scaled.columns if col not in ['cluster_id', 'color', 'FIPS']]

# Calculate the mean and standard deviation for each cluster
mean_values = df_scaled.groupby('cluster_id')[numeric_columns].mean()
std_values = df_scaled.groupby('cluster_id')[numeric_columns].std()  # Calculate standard deviation
count_values = df_scaled.groupby('cluster_id')[numeric_columns].count()  # Count values per cluster
standard_error = std_values / np.sqrt(count_values)  # Calculate standard error

# Calculate the overall mean for each cluster and sort clusters accordingly
mean_values['overall_mean'] = mean_values.mean(axis=1)
mean_values = mean_values.sort_values(by='overall_mean', ascending=False)

# Plot preparation
plt.figure(figsize=(12, 8))

# Color selection based on the 'color' column
colors = df_scaled[['cluster_id', 'color']].drop_duplicates().set_index('cluster_id')['color']

# Create the line plot with the corresponding colors
for cluster in mean_values.index:
    # Get the color for the current cluster
    color = colors.get(cluster, 'gray')  # Use the color or default to 'gray' if no color is found
    
    # Plot the mean values
    plt.plot(mean_values.columns[:-1], mean_values.loc[cluster, mean_values.columns[:-1]], marker='o', 
             label=f'Cluster {cluster}', color=color)
    
    # Add the confidence interval (e.g., 95% CI)
    ci_upper = mean_values.loc[cluster, mean_values.columns[:-1]] + 1.96 * standard_error.loc[cluster]
    ci_lower = mean_values.loc[cluster, mean_values.columns[:-1]] - 1.96 * standard_error.loc[cluster]
    
    # Check dimensions
    if ci_upper.shape[0] == mean_values.columns[:-1].shape[0]:
        # Fill the area for the confidence interval
        plt.fill_between(mean_values.columns[:-1], ci_lower, ci_upper, color=color, alpha=0.2)
    else:
        print(f"Dimension mismatch for cluster {cluster}: ci_upper {ci_upper.shape}, mean_values {mean_values.columns[:-1].shape}")

# Adjust the plot
plt.title('Mean Values of Features by Cluster with Confidence Intervals (Ranked by Overall Mean)')
plt.xlabel('Features')
plt.ylabel('Mean Value')
plt.xticks(rotation=90)  # For better readability of X-axis labels
# Add legend and place it below the plot
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1), ncol=3)
plt.grid()
plt.tight_layout()
plt.show()

# Output the clusters in descending order of their overall mean
print("Cluster Ranking by Overall Mean:")
print(mean_values[['overall_mean']])


## Feature importance

The code trains a separate Random Forest model for each unique cluster to assess the importance of various features in distinguishing each cluster.  
The results are visualized in a heatmap, displaying the significance of each feature across different clusters.


In [ ]:
# Remove the columns 'cluster_id', 'color', 'FIPS'
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns

X = df_scaled[df_scaled.columns[1:-2]]
clusters = df_scaled['cluster_id'].unique()

# Dictionary to store feature importances for each cluster
feature_importances = []

# Iterate over each cluster
for cluster in clusters:
    # Create the target variable: 1 for the current cluster, 0 for all others
    y = (df_scaled['cluster_id'] == cluster).astype(int)
    
    # Train a Random Forest for the binary problem
    model = RandomForestClassifier()
    model.fit(X, y)
    
    # Get feature importances
    importances = model.feature_importances_
    
    # Store the results in the list as a DataFrame
    for feature, importance in zip(X.columns, importances):
        feature_importances.append({'Cluster': cluster, 'Feature': feature, 'Importance': round(importance, 3)})

# Convert the results into a DataFrame
importance_df = pd.DataFrame(feature_importances)

# Create a pivot table for the heatmap
pivot = importance_df.pivot(index="Feature", columns="Cluster", values="Importance")

# Create the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(pivot, cmap="YlGnBu", annot=True, cbar_kws={'label': 'Feature Importance'}, 
            fmt=".3f", annot_kws={"size": 9})  # fmt for decimals and annot_kws for smaller font size
plt.title('Feature Importance Heatmap by Cluster', fontsize=12)
plt.xlabel('Cluster ID', fontsize=10)
plt.ylabel('Features', fontsize=10)
plt.xticks(rotation=45, fontsize=8)
plt.yticks(rotation=0, fontsize=8)
plt.tight_layout()
plt.show()

# Overview

In [ ]:
clusters_df = filtered_county_shape_north_america_merged[['FIPS', 'cluster_id', 'color']]

In [ ]:
# Remove the columns 'cluster_id', 'color', 'FIPS'
X = df_scaled[df_scaled.columns[1:-2]]
clusters = df_scaled['cluster_id'].unique()

# Dictionary to store feature importances for each cluster
feature_importances = []

# Iterate over each cluster and calculate feature importances
for cluster in clusters:
    y = (df_scaled['cluster_id'] == cluster).astype(int)
    model = RandomForestClassifier()
    model.fit(X, y)
    importances = model.feature_importances_
    
    for feature, importance in zip(X.columns, importances):
        feature_importances.append({'Cluster': cluster, 'Feature': feature, 'Importance': round(importance, 3)})

importance_df = pd.DataFrame(feature_importances)

# Extract the top-3 features for each cluster
top_features_by_cluster = (
    importance_df.groupby("Cluster")
    .apply(lambda x: x.nlargest(5, 'Importance'))
    .reset_index(drop=True)
)

# Loop through clusters for scatter, map, and t-SNE with textbox for top-3 features
for cluster in clusters:
    cluster_data = filtered_county_shape_north_america_merged[filtered_county_shape_north_america_merged['cluster_id'] == cluster]
    cluster_shape_data = filtered_county_shape_north_america_merged[filtered_county_shape_north_america_merged['cluster_id'] == cluster]

    fig, (ax2, ax3) = plt.subplots(1, 2, figsize=(20, 7), gridspec_kw={'width_ratios': [6, 4]})
    
    # Map plot
    cluster_shape_data.plot(facecolor=cluster_shape_data['color'], linewidth=0.8, ax=ax2, edgecolor='0.8', legend=False)
    filtered_county_shape_north_america.plot(ax=ax2, color='none', edgecolor='gray', linewidth=0.4, alpha=0.5)
    filtered_state_shape_north_america.plot(ax=ax2, color='none', edgecolor='black', linewidth=0.5)
    count = cluster_counts.get(cluster, 0)
    if cluster == -1:
        legend_elements = [Patch(facecolor='gray', label=f'Noise ({count})')]
    else:
        color = cluster_shape_data['color'].values[0]
        legend_elements = [Patch(facecolor=color, label=f'Cluster {cluster} ({count})')]
    ax2.legend(handles=legend_elements, title='Cluster ID', loc='lower left')
    ax2.set_title(f'Cluster {cluster} Assignment to Counties')
    ax2.axis('off')

    # Add Top-3 Features Textbox
    top_features = top_features_by_cluster[top_features_by_cluster['Cluster'] == cluster]
    feature_text = '\n'.join([f"{row['Feature']}: {row['Importance']}" for _, row in top_features.iterrows()])
    ax2.text(0.95, 0.35, f"Top Features:\n{feature_text}", transform=ax2.transAxes, ha="center", va="top",
             bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="white"))

    # t-SNE plot for cluster
    for label in clusters:
        if label == cluster:
            ax3.scatter(X_tsne_3[cluster_labels == label, 0], X_tsne_3[cluster_labels == label, 1], 
                        color=color_dict[label], label=f'Cluster {label} ({cluster_counts[label]})', alpha=0.75, s=10)
        else:
            ax3.scatter(X_tsne_3[cluster_labels == label, 0], X_tsne_3[cluster_labels == label, 1], 
                        color="gray", alpha=0.2, s=10)
    
    ax3.set_title(f't-SNE Visualization - Cluster {cluster} Highlighted')
    ax3.set_xlabel('t-SNE 1')
    ax3.set_ylabel('t-SNE 2')
    ax3.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3)

    plt.tight_layout()
    plt.show()
